In [1]:
!pip install datasets transformers sentencepiece sacrebleu==1.5.1

     |████████████████████████████████| 311 kB 8.5 MB/s 
     |████████████████████████████████| 3.4 MB 66.1 MB/s 
     |████████████████████████████████| 1.2 MB 58.1 MB/s 
     |████████████████████████████████| 54 kB 3.6 MB/s 
     |████████████████████████████████| 1.1 MB 58.5 MB/s 
     |████████████████████████████████| 133 kB 67.6 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 243 kB 40.7 MB/s 
     |████████████████████████████████| 3.3 MB 53.0 MB/s 
     |████████████████████████████████| 895 kB 64.8 MB/s 
     |████████████████████████████████| 596 kB 73.8 MB/s 
     |████████████████████████████████| 271 kB 74.5 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 56.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
from datasets import load_dataset, load_metric

In [2]:
mk_ro_train = load_dataset("setimes", "mk-ro", split='train[0:10000]')
mk_ro_val = load_dataset("setimes", "mk-ro", split='train[10000:12500]')
mk_ro_test = load_dataset("setimes", "mk-ro", split='train[12500:15000]')

Reusing dataset setimes (/root/.cache/huggingface/datasets/setimes/mk-ro/1.0.0/5b0222bb707caa9d423c61813ef94861e1ccdf82fa4b0bdf4a98de3c9fd33d0d)
Reusing dataset setimes (/root/.cache/huggingface/datasets/setimes/mk-ro/1.0.0/5b0222bb707caa9d423c61813ef94861e1ccdf82fa4b0bdf4a98de3c9fd33d0d)
Reusing dataset setimes (/root/.cache/huggingface/datasets/setimes/mk-ro/1.0.0/5b0222bb707caa9d423c61813ef94861e1ccdf82fa4b0bdf4a98de3c9fd33d0d)


In [3]:
mk_ro_train

Dataset({
    features: ['id', 'translation'],
    num_rows: 10000
})

In [4]:
mk_ro_val

Dataset({
    features: ['id', 'translation'],
    num_rows: 2500
})

In [5]:
mk_ro_test

Dataset({
    features: ['id', 'translation'],
    num_rows: 2500
})

In [6]:
mk_ro_train["translation"][:2]

[{'mk': 'Продолжува кипарскиот застој, додека претседавањето со ЕУ се наближува',
  'ro': 'Impasul din Cipru continuă în timp ce preşedinţia UE se apropie'},
 {'mk': 'Додека ОН ги притиска кипарските Грци и Турци да најдат решение за поделениот остров, малкумина очекуваат чудо пред кипарските Грци да го преземат претседавањето со ЕУ во јули.',
  'ro': 'În timp ce ONU îi presează pe ciprioţii greci şi turci să găsească o soluţie pentru insula divizată, puţini se aşteaptă la un miracol înainte ca ciprioţii greci să preia preşedinţia UE în iulie.'}]

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ro")

In [8]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "mk"
target_lang = "ro"

def preprocess_function(sentences):
    inputs = [prefix + sentence[source_lang] for sentence in sentences["translation"]]
    targets = [sentence[target_lang] for sentence in sentences["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
preprocess_function(mk_ro_train[:2])

{'input_ids': [[51409, 54126, 16974, 14207, 13303, 45931, 18255, 15963, 15138, 15, 14440, 11272, 29426, 37639, 8379, 14440, 11272, 14207, 21208, 55271, 32023, 14207, 55664, 3, 15, 16974, 14207, 16974, 11829, 49625, 53927, 54352, 8379, 11829, 16974, 15138, 15963, 15138, 55729, 11829, 53372, 15, 8379, 14207, 15, 41726, 52502, 15, 8379, 11829, 53815, 32648, 51107, 45931, 18255, 15963, 15138, 0], [15, 39202, 14207, 16974, 11829, 49625, 15, 31595, 39759, 15, 29371, 11272, 53927, 11272, 21208, 11272, 8379, 49625, 15, 14440, 11272, 29426, 37639, 8379, 14440, 11272, 49466, 15, 45930, 18554, 38274, 11272, 15, 11272, 15, 39721, 18255, 18554, 38274, 11272, 15, 16974, 15138, 53815, 55664, 16974, 15138, 21208, 15, 18554, 11829, 49350, 33480, 11272, 11829, 55271, 15, 29426, 14207, 16974, 11829, 13303, 33480, 11272, 14207, 21208, 15, 14207, 32023, 18554, 33725, 3, 15, 54631, 13303, 14440, 18255, 18004, 11272, 29546, 15, 14207, 37205, 11829, 14440, 18255, 15963, 15138, 15138, 21208, 15, 37205, 18255, 

In [9]:
tokenized_train = mk_ro_train.map(preprocess_function, batched=True)
tokenized_val = mk_ro_val.map(preprocess_function, batched=True)
tokenized_test = mk_ro_test.map(preprocess_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/setimes/mk-ro/1.0.0/5b0222bb707caa9d423c61813ef94861e1ccdf82fa4b0bdf4a98de3c9fd33d0d/cache-ecaf7c0454fd5a59.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/setimes/mk-ro/1.0.0/5b0222bb707caa9d423c61813ef94861e1ccdf82fa4b0bdf4a98de3c9fd33d0d/cache-9d391781f9698952.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/setimes/mk-ro/1.0.0/5b0222bb707caa9d423c61813ef94861e1ccdf82fa4b0bdf4a98de3c9fd33d0d/cache-842a5a03915c644a.arrow


In [10]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ro")

In [11]:
batch_size = 16
model_name = "opus-mt-en-ro"

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True    
)

metric = load_metric("sacrebleu")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {
        "bleu": result["score"] 
    }

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]

    result["gen_len"] = np.mean(prediction_lens)
    result = {
        k: round(v, 4) for k, v in result.items()
    }
    return result

In [13]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: id, translation.
***** Running training *****
  Num examples = 10000
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6250


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,3.624200,2.813786,9.383200,36.864800
2,2.840200,2.462099,14.281800,34.660000
3,2.475600,2.291636,17.470200,34.253200
4,2.101200,2.195141,19.441300,33.962400
5,1.954600,2.130142,21.052600,33.516400
6,1.857500,2.084888,22.085000,33.176800
7,1.787900,2.065709,22.751500,32.690800
8,1.677700,2.048159,23.498100,32.991600
9,1.636800,2.038754,23.708700,32.846400
10,1.622300,2.036164,23.662100,32.860000


Saving model checkpoint to opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-500
Configuration saved in opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-500/config.json
Model weights saved in opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-500/tokenizer_config.json
Special tokens file saved in opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: id, translation.
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 16
Saving model checkpoint to opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-1000
Configuration saved in opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-1000/config.json
Model weights saved in opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-1000/to

TrainOutput(global_step=6250, training_loss=2.1104263427734375, metrics={'train_runtime': 3843.2517, 'train_samples_per_second': 26.02, 'train_steps_per_second': 1.626, 'total_flos': 3389645814497280.0, 'train_loss': 2.1104263427734375, 'epoch': 10.0})

In [17]:
from transformers import MarianMTModel, MarianTokenizer

# src_text = ['Во таа група се наоfаат пеесет и пет земји, вклучувајќи ги и сите земји-членки на ЕУ.']
src_text = ['Хрватска е рангирана на 48-мото место со HDI од 0,830, и е единствена земја од Југоисточна Европа која се наоfа во групата земји со висок човеков развој.']
model_name = 'opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000'

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

Didn't find file opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/added_tokens.json. We won't load it.
Didn't find file opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/tokenizer.json. We won't load it.
loading file opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/source.spm
loading file opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/target.spm
loading file opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/vocab.json
loading file opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/tokenizer_config.json
loading file None
loading file opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/special_tokens_map.json
loading file None
loading configuration file opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/config.json
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-en-ro",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_drop

['Croaţia s-a situat pe locul 48 cu HID de 0,30 şi este o ţară vecină din Europa de Sud-est, care se află în ţările din grup cu un nivel înalt de dezvoltare.']

In [17]:
mk_ro_test["translation"][:5]

[{'mk': 'Хрватска е рангирана на 48-мото место со HDI од 0,830, и е единствена земја од Југоисточна Европа која се наоfа во групата земји со висок човеков развој.',
  'ro': 'Croaţia, plasată pe locul al 48-lea cu un HDI de 0,830, este singura ţară sud-est europeană din grupul dezvoltării umane înalte.'},
 {'mk': 'Во таа група се наоfаат пеесет и пет земји, вклучувајќи ги и сите земји-членки на ЕУ.',
  'ro': 'Cincizeci şi cinci de ţări se află în acest grup, inclusiv toate statele membre UE.'},
 {'mk': 'Другите земји од Југоисточна Европа се рангирани во групата средно развиени земји (HDI од 0,500 до 0,799).',
  'ro': 'Celelalte ţări sud-est europene sunt plasate în grupul de dezvoltare umană medie-înaltă (HDI de la 0,500 la 0,799).'},
 {'mk': 'Бугарија се наоfа на 56-тото место со HDI од 0,796, по што следи Македонија, која е рангирана на 60-тото место со HDI од 0,793.',
  'ro': 'Bulgaria se plasează pe locul al 56-lea cu un HDI de 0,796, fiind urmată de Macedonia, clasată a 60-a cu un

In [15]:
!zip -r opus-mt-en-ro-finetuned-mk-to-ro.zip opus-mt-en-ro-finetuned-mk-to-ro/

  adding: opus-mt-en-ro-finetuned-mk-to-ro/ (stored 0%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/ (stored 0%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/tokenizer_config.json (deflated 40%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/vocab.json (deflated 70%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/pytorch_model.bin (deflated 7%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/training_args.bin (deflated 49%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/optimizer.pt (deflated 8%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/config.json (deflated 60%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/trainer_state.json (deflated 78%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/special_tokens_map.json (deflated 34%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/target.spm (deflated 49%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/s

In [18]:
!zip -r opus-mt-en-ro-finetuned-mk-to-ro_checkpoint-6000.zip opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/

  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/ (stored 0%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/tokenizer_config.json (deflated 40%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/vocab.json (deflated 70%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/pytorch_model.bin (deflated 7%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/training_args.bin (deflated 49%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/optimizer.pt (deflated 8%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/config.json (deflated 60%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/trainer_state.json (deflated 78%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/special_tokens_map.json (deflated 34%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/target.spm (deflated 49%)
  adding: opus-mt-en-ro-finetuned-mk-to-ro/checkpoint-6000/source.spm (deflated 48%)
  adding: opus-mt-en-ro-finetun

In [ ]:
# Bg -> En -> Mk -> Ro
# Grk -> En -> Mk -> Ro
# Sr -> En -> Mk -> Ro